# Craigslist Vehicles — Initial Data Checks

This notebook performs preliminary profiling on the `vehicles.csv` dataset to understand schema, data quality, duplicates, and outlier behavior.

In [1]:
from pathlib import Path
import csv

DATA_PATH = Path('data/vehicles.csv')
with DATA_PATH.open() as f:
    reader = csv.DictReader(f)
    records = list(reader)

len(records)

6


In [2]:
records[0]

{'id': '1', 'url': 'https://example.com/1', 'region': 'seattle', 'region_url': 'https://seattle.craigslist.org', 'price': '8500', 'year': '2012', 'manufacturer': 'honda', 'model': 'civic lx', 'condition': 'good', 'cylinders': '4 cylinders', 'fuel': 'gas', 'odometer': '120000', 'title_status': 'clean', 'transmission': 'automatic', 'VIN': '1HGCM82633A123456', 'drive': 'fwd', 'size': 'compact', 'type': 'sedan', 'paint_color': 'blue', 'image_url': 'https://images.craigslist.org/1', 'description': 'Reliable daily driver with maintenance records.', 'county': 'king county', 'state': 'wa', 'lat': '47.6062', 'long': '-122.3321', 'posting_date': '2021-05-15T10:30:00-0700'}

In [3]:
# Summaries: inferred dtypes, unique counts, and missingness
from collections import defaultdict


def infer_type(values):
    types = set()
    for v in values:
        if v == '' or v is None:
            continue
        try:
            int(v)
        except ValueError:
            try:
                float(v)
            except ValueError:
                types.add('object')
            else:
                types.add('float')
        else:
            types.add('int')
    if 'object' in types:
        return 'object'
    if 'float' in types and 'int' in types:
        return 'float'
    if 'float' in types:
        return 'float'
    if 'int' in types:
        return 'int'
    return 'object'

columns = list(records[0].keys())
values_by_col = {col: [row[col] for row in records] for col in columns}
unique_counts = {col: len({v for v in vals}) for col, vals in values_by_col.items()}
missing_counts = {col: sum(1 for v in vals if v == '' or v is None) for col, vals in values_by_col.items()}
missing_pct = {col: (missing_counts[col] / len(records)) * 100 for col in columns}
dtypes = {col: infer_type(vals) for col, vals in values_by_col.items()}


def format_table(rows, headers):
    widths = [len(h) for h in headers]
    for row in rows:
        for i, cell in enumerate(row):
            widths[i] = max(widths[i], len(str(cell)))
    lines = []
    header_line = ' | '.join(str(h).ljust(widths[i]) for i, h in enumerate(headers))
    separator = '-+-'.join('-' * widths[i] for i in range(len(headers)))
    lines.append(header_line)
    lines.append(separator)
    for row in rows:
        lines.append(' | '.join(str(cell).ljust(widths[i]) for i, cell in enumerate(row)))
    print('\n'.join(lines))

rows = [
    (col, dtypes[col], unique_counts[col], missing_counts[col], f"{missing_pct[col]:.1f}%")
    for col in columns
]
format_table(rows, ['column', 'dtype', 'unique', 'missing', 'missing_pct'])


column       | dtype  | unique | missing | missing_pct
-------------+--------+--------+---------+------------
id           | int    | 6      | 0       | 0.0%       
url          | object | 6      | 0       | 0.0%       
region       | object | 4      | 0       | 0.0%       
region_url   | object | 4      | 0       | 0.0%       
price        | int    | 5      | 0       | 0.0%       
year         | int    | 5      | 0       | 0.0%       
manufacturer | object | 5      | 0       | 0.0%       
model        | object | 5      | 0       | 0.0%       
condition    | object | 4      | 0       | 0.0%       
cylinders    | object | 4      | 0       | 0.0%       
fuel         | object | 2      | 0       | 0.0%       
odometer     | int    | 5      | 0       | 0.0%       
title_status | object | 2      | 0       | 0.0%       
transmission | object | 2      | 0       | 0.0%       
VIN          | object | 5      | 0       | 0.0%       
drive        | object | 2      | 0       | 0.0%       
size      

In [4]:
# Duplicate checks across key identifiers
from collections import defaultdict


def find_duplicates(key):
    idx = defaultdict(list)
    for i, row in enumerate(records):
        value = row[key]
        if value:
            idx[value].append(i)
    return {k: v for k, v in idx.items() if len(v) > 1}

for key in ['id', 'url', 'VIN']:
    dupes = find_duplicates(key)
    if dupes:
        print(f"{key}: {len(dupes)} duplicate group(s) -> {sorted(dupes.items())}")
    else:
        print(f"{key}: no duplicates detected")


id: no duplicates detected
url: no duplicates detected
VIN: 1 duplicate group(s) -> [('1HGCM82633A123456', [0, 5])]


In [5]:
# Suggested deduplication strategy based on VIN duplicates
print('Drop secondary entries sharing the same VIN while preserving the earliest listing by posting_date or lowest row index.')


Drop secondary entries sharing the same VIN while preserving the earliest listing by posting_date or lowest row index.


In [6]:
# Outlier diagnostics for key numeric columns
from statistics import mean


def parse_float_list(values):
    parsed = []
    for v in values:
        if v == '' or v is None:
            continue
        parsed.append(float(v))
    return parsed


def quantiles(values, probs):
    values = sorted(values)
    n = len(values)
    results = []
    for p in probs:
        pos = (n - 1) * (p / 100)
        lower = int(pos)
        upper = min(lower + 1, n - 1)
        weight = pos - lower
        results.append(values[lower] * (1 - weight) + values[upper] * weight)
    return results

for col in ['price', 'year', 'odometer', 'lat', 'long']:
    series = parse_float_list(values_by_col[col])
    q5, q25, q50, q75, q95 = quantiles(series, [5, 25, 50, 75, 95])
    print(
        f"{col}: min={min(series):.2f}, max={max(series):.2f}, mean={mean(series):.2f}, "
        f"p05={q5:.2f}, p25={q25:.2f}, p50={q50:.2f}, p75={q75:.2f}, p95={q95:.2f}"
    )


price: min=2300.00, max=999999.00, mean=179466.50, p05=3850.00, p25=8500.00, p50=10500.00, p75=36875.00, p95=761249.25
year: min=1985.00, max=2020.00, mean=2007.17, p05=1988.50, p25=2002.25, p50=2012.00, p75=2014.25, p95=2018.75
odometer: min=15000.00, max=220000.00, mean=100833.33, p05=23750.00, p25=57500.00, p50=100000.00, p75=120000.00, p95=195000.00
lat: min=34.05, max=47.62, mean=43.69, p05=35.47, p25=41.18, p50=46.56, p75=47.61, p95=47.62
long: min=-122.68, max=-104.99, mean=-118.82, p05=-122.60, p25=-122.34, p50=-122.33, p75=-119.27, p95=-108.30


The extreme price value ($999,999) and the duplicate VIN indicate potential data-quality issues requiring manual review or rule-based filtering prior to modeling.